Code based on the Web example from https://github.com/nmslib/hnswlib
Uses the precomputed Euc data with known NNs

In [1]:
import numpy as np
import hnswlib
from numpy import genfromtxt

In [2]:
# Read in the data from Euc 10,000

data = genfromtxt('data/euc_data_20_10000.csv', delimiter=',')
queries = genfromtxt('data/euc_queries_20_10000.csv', delimiter=',')
data_nn_indices = genfromtxt('data/data_indices_nns_20_10000_100.csv', delimiter=' ')
data_nn_dists = genfromtxt('data/data_dists_nns_20_10000_100.csv', delimiter=' ')
query_nn_indices = genfromtxt('data/query_indices_nns_20_1000_100.csv', delimiter=' ')
query_nn_dists = genfromtxt('data/query_dists_nns_20_1000_100.csv', delimiter=' ')

num_elements = data.shape[0]
dim = data.shape[1]
ids = np.arange(num_elements)


In [3]:
print( f"Dims = {dim}")
print( f"Num of elements = {num_elements}")
print( f"Shape of ids = {ids.shape}" )
print( f"Shape of data = {data.shape}" )
print( f"Shape of queries = {queries.shape}")
print( f"Shape of nn_indices = {data_nn_indices.shape}" )
print( f"Shape of nn dists = {data_nn_dists.shape}")
print( f"Shape of nn_queries = {query_nn_indices.shape}" )
print( f"Shape of nn query dists = {query_nn_dists.shape}")


Dims = 20
Num of elements = 10000
Shape of ids = (10000,)
Shape of data = (10000, 20)
Shape of queries = (1000, 20)
Shape of nn_indices = (10000, 100)
Shape of nn dists = (10000, 100)
Shape of nn_queries = (1000, 100)
Shape of nn query dists = (1000, 100)


In [4]:
# Declaring index
p = hnswlib.Index(space = 'l2', dim = dim) # possible options are l2, cosine or ip

# Initializing index - the maximum number of elements should be known beforehand
p.init_index(max_elements = num_elements, ef_construction = 200, M = 16) # these params from Web site

# Element insertion (can be called several times):
p.add_items(data, ids)

In [5]:
#Query the index
num_queries = 100;
num_nns = 10;

# Query dataset, k - number of the closest elements (returns 2 numpy arrays)
labels, distances = p.knn_query(queries[:num_queries], k = num_nns)  # 10 NNs for first 10 queries

In [6]:
# and print the true NN 10 labels
# as above the first num_queries and num_nns
true_query_nn_indices_subset = query_nn_indices[:num_queries,0:num_nns]
true_query_nn_indices_subset = true_query_nn_indices_subset.astype(int)


In [7]:
# as above the first num_queries and num_nns
true_query_nn_dists_subset = query_nn_dists[:num_queries,0:num_nns] 
# square the gt to match the metric they use
squared_query_nn_dists = np.square(true_query_nn_dists_subset)

In [8]:
import math

def l2_squared(p1: np.array, p2: np.array):
    distances = np.sum((p1 - p2) ** 2)
    return distances

def euc_scalar(p1: np.array, p2: np.array):
    distances = math.sqrt(np.sum(np.square((p1 - p2))))
    return distances

In [9]:
# Do a santity check on distances - Not needed for any future computation
query_index = 2
result_index = 3
query_row = queries[query_index,:]
true_nn_index = true_query_nn_indices_subset[query_index,result_index]
true_nn_dist = true_query_nn_dists_subset[query_index,result_index]
label_index = labels[query_index,result_index]

print( f"Query index = {query_index} result index = {result_index}")
print( f"Index of first result from file {true_nn_index}" )
print( f"Ground truth distances to first result from file {true_nn_dist}" )
print( f"Ground truth squared distances to first result from file {true_nn_dist ** 2}" )

print( f"Recomputed GT distance {euc_scalar( data[true_nn_index,:],query_row)}" )
print( f"Recomputed GT squared distance {l2_squared(data[true_nn_index],query_row)}" )

print( f"Index of first result calculated by HNSW {label_index}" )
print( f"HNSW distance to first result {distances[query_index,result_index]}")
print( f"Recomputed Distance from first query to HNSW result {l2_squared( data[label_index,:],query_row)}" )


Query index = 2 result index = 3
Index of first result from file 2711
Ground truth distances to first result from file 1.2499
Ground truth squared distances to first result from file 1.56225001
Recomputed GT distance 1.2498795838156396
Recomputed GT squared distance 1.5621989740391562
Index of first result calculated by HNSW 8355
HNSW distance to first result 1.6411247253417969
Recomputed Distance from first query to HNSW result 1.6411247289744528


In [15]:
# Do some comparisons of distances and true_nn_dists

for row in range(distances.shape[0]):
    hnsw_index = labels[row]
    print( f"hnsw labels = {hnsw_index}" )
    true_indices = true_query_nn_indices_subset[row]
    print( f"true labels = {true_indices}" )
    hnsw_row = distances[row]
    print( f"hnsw dists = {hnsw_row}" )
    hnsw_sums = np.sum( hnsw_row )
    true_row = squared_query_nn_dists[row]    # true_nn_dists_subset
    print( f"true dists = {true_row}" )
    true_sums = np.sum(true_row) 
    print( f"Ave HNSW row dist = {hnsw_sums / num_nns} ave true sum = {true_sums / num_nns }\n--------" )


hnsw labels = [ 937 2527 4629  605 7516  811 3027 4306 2464 6487]
true labels = [1440  937 2527 4629  605 7516  811 3682 3027 2404]
hnsw dists = [1.0469908 1.1044914 1.1362563 1.1960988 1.2354085 1.237374  1.2476412
 1.2667387 1.2684094 1.2878232]
true dists = [0.77118255 1.04693824 1.10439081 1.136356   1.19617969 1.23543225
 1.23743376 1.24612569 1.247689   1.25776225]
Ave HNSW row dist = 1.2027233123779297 ave true sum = 1.1479490238900003
--------
hnsw labels = [9374 5618 7892 5570 3314 8287 7365 1438 3916 6255]
true labels = [9374 5618 7892 5570 3314 8287 7365 1438 6450 7323]
hnsw dists = [0.76676977 0.80250406 0.80915487 0.9901552  1.0069366  1.0244617
 1.0402162  1.053457   1.0824888  1.1059779 ]
true dists = [0.76676292 0.80251139 0.80915422 0.9901643  1.00701225 1.02454884
 1.04019601 1.05349696 1.065024   1.07371044]
Ave HNSW row dist = 0.9682122230529785 ave true sum = 0.9632581337199999
--------
hnsw labels = [2600 7764 2711 8355 3134 2774 9618  129 4742 7281]
true labels =